In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.spatial.distance as dist
import matplotlib as mpl

import iblofunmatch.inter as ibfm
import os

from navground import core, sim

import perdiver.perdiver as perdiver
from perdiver.distances import *

plots_dir = "plots/corridor/"
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)
os.makedirs(plots_dir, exist_ok=True)

In [ ]:
length = 8.0
num_steps = 2000
width=1.0
# num_agents = 28
num_agents = 20
# num_steps = 0
shift_time = 100
# shift_time = 50
steps_list = list(range(0, num_steps-shift_time, shift_time))
diags_list = []
weight= 3
behaviour_list = ["ORCA", "HL"]
for behaviour in behaviour_list:
    yaml = f"""
    steps: {num_steps}
    time_step: 0.1
    save_directory: ''
    record_pose: true
    record_twist: true
    scenario:
      type: Corridor
      length: {length}
      width: {width} 
      groups:
        -
          type: thymio
          number: {num_agents}
          radius: 0.08
          control_period: 0.1
          speed_tolerance: 0.02
          kinematics:
            type: 2WDiff
            wheel_axis: 0.094
            max_speed: 0.166
          behavior:
            type: {behaviour}
            optimal_speed: 0.12
            horizon: 5.0
            safety_margin: 0.034
          state_estimation:
            type: Bounded
            range: 5.0
    """
    experiment = sim.load_experiment(yaml)
    experiment.run()
    run = experiment.runs[0]
    ps = run.poses[:,:,[0,1]]
    twists = run.twists[:,:,:2] # ignore angular speeds
    # COmpute persistence diagrams and store
    diags = []
    for idx, start_step in enumerate(steps_list):
        Dist_X, Dist_Y, Dist_Z = compute_distance_matrices_trajectories_corridor(ps, twists, start_step, shift_time, weight, length)
        diags.append(perdiver.get_matching_diagram(Dist_X, Dist_Y, Dist_Z, output_dir))
    diags_list.append(diags)

Compute persistence images

In [ ]:
from gudhi import representations
npixels = 30
xmax = max(np.max(diags_list[0]), np.max(diags_list[1]))*1.1
ymax = xmax
perim = representations.PersistenceImage(resolution=[npixels, npixels], bandwidth=0.08, im_range=[0, xmax, -ymax, 0])
perim.fit(diags_list[0] + diags_list[1])
ORCA_perimages = perim.transform(diags_list[0])
HL_perimages = perim.transform(diags_list[1])

In [ ]:
xmax

In [ ]:
for i, (image, diag) in enumerate(zip(ORCA_perimages, diags_list[0])):
    fig, ax = plt.subplots(ncols=2, figsize=(8,4))
    image = image.reshape(npixels,-1)
    ax[0].imshow(image)
    ax[0].set_xticks([])
    ax[0].set_yticks([])
    perdiver.plot_matching_diagram(diag, ax[1], max_val_diag=xmax)
    fig.suptitle(f"{behaviour_list[0]} step: {steps_list[i]:5d}")
    plt.tight_layout()
    plt.savefig(plots_dir + f"ORCA_{steps_list[i]:04d}" + ".png")
    plt.close()

In [ ]:
for i, (image, diag) in enumerate(zip(HL_perimages, diags_list[1])):
    fig, ax = plt.subplots(ncols=2, figsize=(8,4))
    image = image.reshape(npixels,-1)
    ax[0].imshow(image)
    ax[0].set_xticks([])
    ax[0].set_yticks([])
    perdiver.plot_matching_diagram(diag, ax[1], max_val_diag=xmax)
    fig.suptitle(f"{behaviour_list[1]} step: {steps_list[i]:5d}")
    plt.tight_layout()
    plt.savefig(plots_dir + f"HL_{steps_list[i]:04d}" + ".png")
    plt.close()

# Plot videos of both experiments to see how they behave

In [ ]:
import matplotlib.colors as colors
import matplotlib.cm as cmx
from navground.sim.ui.video import record_video

def linear_map(a, b, cmap):
    c = cmx.ScalarMappable(norm=colors.Normalize(vmin=a, vmax=b), cmap=cmap)  
    def f(v):
        r, g, b, _ = c.to_rgba(v)
        return f"#{int(r * 255):02x}{int(g * 255):02x}{int(b * 255):02x}"
    return f


fill_map = linear_map(0.0, 1.0, cmap=cmx.RdYlGn)

def f(entity):
    if isinstance(entity, sim.Agent):
        return {'fill': fill_map(entity.behavior.efficacy)}
    return {}
    

In [ ]:
for behaviour in ["ORCA", "HL"]:
    yaml = f"""
    steps: 2
    time_step: 0.1
    save_directory: ''
    record_pose: true
    record_twist: true
    scenario:
      type: Corridor
      length: {length}
      width: {width} 
      groups:
        -
          type: thymio
          number: {num_agents}
          radius: 0.08
          control_period: 0.1
          speed_tolerance: 0.02
          kinematics:
            type: 2WDiff
            wheel_axis: 0.094
            max_speed: 0.166
          behavior:
            type: {behaviour}
            optimal_speed: 0.12
            horizon: 5.0
            safety_margin: 0.034
          state_estimation:
            type: Bounded
            range: 5.0
    """
    experiment = sim.load_experiment(yaml)
    experiment.run()
    run = experiment.runs[0]
    # Record video
    record_video(f"{plots_dir}{behaviour}_exp_{num_agents}.mp4", run.world, time_step=0.1, duration=200.0, factor=14.0,
                  bounds=((0, 0), (length, width)), decorate=f, width=700, display_shape=True)

In [ ]:
from navground.sim.ui.render import png_for_world
from IPython.display import Image

world = run.world
# Image(data=png_for_world(world), width=700, filename="corridor.png")
with open("corridor.png", "wb") as png:
    png.write(png_for_world(world))

Now, we can see the generated videos

In [ ]:
from IPython.display import Video

Video(f"{plots_dir}ORCA_exp_{num_agents}.mp4", width=700)

In [ ]:
from IPython.display import Video

Video(f"{plots_dir}HL_exp_{num_agents}.mp4", width=700)